In [1]:
%reset -f
# use this to make skin files, comment out certain lines for specific skim rules to apply 
import ROOT
import array
import numpy as np
from datetime import datetime
import traceback
# file = ROOT.TFile.Open("/w/hallb-scshelf2102/clas12/ricardos/testinggroovy/ePip.wCentral.pass2.inb.qa.Sp19.rec_clas_006648.evio.00000-00004.hipo.root")
# file = ROOT.TFile.Open("/w/hallb-scshelf2102/clas12/ricardos/testinggroovy/g1_all_data.root")
file = ROOT.TFile.Open("/w/hallb-scshelf2102/clas12/ricardos/Momentum_Corrections/Central_Tracking/Spring2019/Inbending/Single_Pion_Channel_epipN/central_only/central_all.root")

tree = file.Get("h22")
irdf = ROOT.RDataFrame(tree)
pipPAxis = "P_{#pi^{+}}"
pipPhiAxis = "#phi_{#pi^{+}}"
pipThetaAxis = "#theta_{#pi^{+}}"
print("Number of entries:", irdf.Count().GetValue())
# com_to_check = 1
f_rdf = irdf

f_rdf = f_rdf.Define("beam", "ROOT::Math::PxPyPzMVector(0, 0, 10.1998, 0)")
f_rdf = f_rdf.Define("targ", "ROOT::Math::PxPyPzMVector(0, 0, 0, 0.938)")

f_rdf = f_rdf.Define("pipc", "ROOT::Math::PxPyPzMVector(pipx, pipy, pipz, 0.13957  )")
f_rdf = f_rdf.Define("elc", "ROOT::Math::PxPyPzMVector(ex, ey, ez, 0)")
f_rdf = f_rdf.Define("o_v", "beam + targ - elc - pipc")
f_rdf = f_rdf.Define("f_o", "o_v.M()")
# f_rdf = f_rdf.Filter("f_o < 1.1")
# f_rdf = f_rdf.Filter("f_o > 0")

p_rdf = f_rdf
p_rdf = p_rdf.Define("pipP", "sqrt(pipx*pipx+pipy*pipy+pipz*pipz)")
p_rdf = p_rdf.Define("pipPhi", "((180/3.1415926)*atan2(pipy, pipx))")
p_rdf = p_rdf.Define("pipTheta", "atan2(sqrt(pipx*pipx+pipy*pipy), pipz)*(180/3.1415926)")
p_rdf = p_rdf.Define("pipPhiw","""
    double pipPhiW;
    if(pipPhi < 30 && component >20){
        pipPhiW = pipPhi + 360;
        }
    else{
        pipPhiW = pipPhi;
        }
    return pipPhiW;
""")
# p_rdf = p_rdf.Define("pipPhil","""
#     double pipPhil;
#     pipPhil = pipPhiw - 7.5*component-9;
#     return pipPhil;
# """)
# f_rdf = p_rdf
# f_rdf = f_rdf.Filter(f"component == {com_to_check}")

# f_rdf = f_rdf
# print("Number of entries:", p_rdf.Count().GetValue())
# print("Number of entries:", f_rdf.Count().GetValue())
# output_file = "skimmed_central_p3_MM_cut.root"
# p_rdf.Snapshot("h22", output_file)
print("Number of entries:", f_rdf.Count().GetValue())
# del p_rdf

# file.Close()

Welcome to JupyROOT 6.22/06
Number of entries: 145133352
Number of entries: 145133352


In [2]:
rdf = p_rdf.Define("el", "sqrt(ex*ex + ey*ey + ez*ez)")
##=====##       Polar Angles       ##=====##
rdf = rdf.Define("elth", "atan2(sqrt(ex*ex + ey*ey), ez)*(180/3.1415926)")
# if("Central" in str(pass_version)):
#     print(color.BOLD, "\nMAKING CUT ON ELECTRON POLAR ANGLE TO SELECT THE EVENTS FROM THE 'Central' DETECTOR\n", color.END)
#     rdf = rdf.Filter("elth > 35 && elth < 135")
#             if("Forward" in str(pass_version)):
#                 print(color.BOLD, "\nMAKING CUT ON ELECTRON POLAR ANGLE TO SELECT THE EVENTS FROM THE 'Forward' DETECTOR\n", color.END)
#                 rdf = rdf.Filter("elth >  5 && elth <  35")
##=====##     Azimuthal Angles     ##=====##
rdf = rdf.Define("elPhi", """
    double elPhi = (180/3.1415926)*atan2(ey, ex);
    if(((esec == 4 || esec == 3) && elPhi < 0) || (esec > 4 && elPhi < 90)){
        elPhi += 360;
    }
    return elPhi;
""")
rdf = rdf.Define("localelPhi", "elPhi - (esec - 1)*60")
rdf = rdf.Define("localelPhiS", "localelPhi - (30/el)")
rdf = rdf.Define("elPhiS", "elPhi - (30/el)")
rdf = rdf.Define("elPhiNS", "(180/3.1415926)*atan2(ey, ex)") # 'NS' ==> No shifts (distribution will be from ±180˚)

In [3]:
Calculated_Exclusive_Cuts = "".join(["""
        auto ele  = ROOT::Math::PxPyPzMVector(ex, ey, ez, 0);
        auto pip0 = ROOT::Math::PxPyPzMVector(pipx, pipy, pipz, 0.13957);
        auto MM_Vector = beam + targ - ele - pip0;
        auto cut_upper = 1.1;
        auto cut_lower = 0;
        // most recent exclusivity cuts 
        if(pipsec == -1){
            if(esec == 1){
                if(localelPhiS > -5 && localelPhiS < 5){
                    // Upper Cut
                    cut_upper = (-0.0592999)*el + (1.5334795);
                    // Lower Cut
                    cut_lower = (0.005)*el + (0.7985106);
                }
                if(localelPhiS < -5){
                    // Upper Cut
                    cut_upper = (-0.0635133)*el + (1.5630763);
                    // Lower Cut
                    cut_lower = (0.005)*el + (0.8075723);
                }
                if(localelPhiS > 5){
                    // Upper Cut
                    cut_upper = (-0.0417367)*el + (1.3671238);
                    // Lower Cut
                    cut_lower = (0.005)*el + (0.799642);
                }
            }
            if(esec == 2){
                if(localelPhiS > -5 && localelPhiS < 5){
                    // Upper Cut
                    cut_upper = (-0.032936)*el + (1.305817);
                    // Lower Cut
                    cut_lower = (1e-05)*el + (0.8250863);
                }
                if(localelPhiS < -5){
                    // Upper Cut
                    cut_upper = (-0.0487001)*el + (1.4357224);
                    // Lower Cut
                    cut_lower = (1e-05)*el + (0.8338142);
                }
                if(localelPhiS > 5){
                    // Upper Cut
                    cut_upper = (-0.0426093)*el + (1.3719635);
                    // Lower Cut
                    cut_lower = (1e-05)*el + (0.8126858);
                }
            }
            if(esec == 3){
                if(localelPhiS > -5 && localelPhiS < 5){
                    // Upper Cut
                    cut_upper = (-0.0509723)*el + (1.4492638);
                    // Lower Cut
                    cut_lower = (1e-05)*el + (0.822748);
                }
                if(localelPhiS < -5){
                    // Upper Cut
                    cut_upper = (-0.045786)*el + (1.3998097);
                    // Lower Cut
                    cut_lower = (1e-05)*el + (0.8193907);
                }
                if(localelPhiS > 5){
                    // Upper Cut
                    cut_upper = (-0.0653454)*el + (1.5571954);
                    // Lower Cut
                    cut_lower = (1e-05)*el + (0.8318872);
                }
            }
            if(esec == 4){
                if(localelPhiS > -5 && localelPhiS < 5){
                    // Upper Cut
                    cut_upper = (-0.0320573)*el + (1.3071999);
                    // Lower Cut
                    cut_lower = (1e-05)*el + (0.847568);
                }
                if(localelPhiS < -5){
                    // Upper Cut
                    cut_upper = (-0.0493433)*el + (1.4431333);
                    // Lower Cut
                    cut_lower = (1e-05)*el + (0.8390728);
                }
                if(localelPhiS > 5){
                    // Upper Cut
                    cut_upper = (-0.0546428)*el + (1.4943016);
                    // Lower Cut
                    cut_lower = (1e-05)*el + (0.8534834);
                }
            }
            if(esec == 5){
                if(localelPhiS > -5 && localelPhiS < 5){
                    // Upper Cut
                    cut_upper = (-0.0329613)*el + (1.3060447);
                    // Lower Cut
                    cut_lower = (0.005)*el + (0.8241186);
                }
                if(localelPhiS < -5){
                    // Upper Cut
                    cut_upper = (-0.0500122)*el + (1.4492993);
                    // Lower Cut
                    cut_lower = (-0.005)*el + (0.9087516);
                }
                if(localelPhiS > 5){
                    // Upper Cut
                    cut_upper = (-0.0496823)*el + (1.4346816);
                    // Lower Cut
                    cut_lower = (0.005)*el + (0.8256886);
                }
            }
            if(esec == 6){
                if(localelPhiS > -5 && localelPhiS < 5){
                    // Upper Cut
                    cut_upper = (-0.0229957)*el + (1.2248551);
                    // Lower Cut
                    cut_lower = (-0.005)*el + (0.8887692);
                }
                if(localelPhiS < -5){
                    // Upper Cut
                    cut_upper = (-0.0377501)*el + (1.3408122);
                    // Lower Cut
                    cut_lower = (-0.0021906)*el + (0.8747312);
                }
                if(localelPhiS > 5){
                    // Upper Cut
                    cut_upper = (-0.0553155)*el + (1.4872476);
                    // Lower Cut
                    cut_lower = (-0.005)*el + (0.8852568);
                }
            }
        }

        return (MM_Vector.M() < cut_upper && MM_Vector.M() > cut_lower);

    """])

In [4]:
rdf = rdf.Define("pipPhis","""
    double pipPhis;
    if(component == 1){
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (1.166e-04*pipTheta*pipTheta+-0.04825*pipTheta+-3.871) * pow( pipP, (-1.611e-07*pipTheta*pipTheta+0.002026*pipTheta+0.1913) )) + (-5.758e-04*pipTheta*pipTheta+-0.06956*pipTheta+1.589) );
    }
    else if(component == 2){
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.242e-04*pipTheta*pipTheta+-0.07664*pipTheta+-3.043) * pow( pipP, (-1.724e-05*pipTheta*pipTheta+0.004304*pipTheta+0.1305) )) + (4.961e-04*pipTheta*pipTheta+-0.2251*pipTheta+13.86) );
    }
    else if(component == 3){    
        pipPhis = pipPhiw - ((-0.08786*pipTheta*pipTheta+23.27*pipTheta+-6.904e+03) * exp( (4.124e-04*pipTheta*pipTheta+-0.08415*pipTheta+-3.433) * pow( pipP, (-2.416e-05*pipTheta*pipTheta+0.00488*pipTheta+0.09128) )) + (5.366e-04*pipTheta*pipTheta+-0.244*pipTheta+21.95) );
    }   
    else if(component == 4){   
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (2.553e-04*pipTheta*pipTheta+-0.05744*pipTheta+-3.906) * pow( pipP, (4.776e-07*pipTheta*pipTheta+8.033e-04*pipTheta+0.2642) )) + (-3.460e-04*pipTheta*pipTheta+-0.03679*pipTheta+19.92) );
    }
    else if(component == 5){   
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.534e-04*pipTheta*pipTheta+-0.07462*pipTheta+-3.597) * pow( pipP, (-1.194e-05*pipTheta*pipTheta+0.002749*pipTheta+0.2301) )) + (-1.717e-04*pipTheta*pipTheta+-0.06218*pipTheta+25.35) );
    }
    else if(component == 6){   
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (7.933e-04*pipTheta*pipTheta+-0.1461*pipTheta+-1.118) * pow( pipP, (-7.550e-05*pipTheta*pipTheta+0.01294*pipTheta+-0.1252) )) + (0.003297*pipTheta*pipTheta+-0.5815*pipTheta+52.54) );
    }
    else if(component == 7){   
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (5.437e-05*pipTheta*pipTheta+-0.05879*pipTheta+-3.322) * pow( pipP, (7.166e-06*pipTheta*pipTheta+0.002839*pipTheta+0.1354) )) + (0.002449*pipTheta*pipTheta+-0.5645*pipTheta+62.28) );
    }
    else if(component == 8){   
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (5.556e-04*pipTheta*pipTheta+-0.09487*pipTheta+-2.566) * pow( pipP, (-3.641e-05*pipTheta*pipTheta+0.005786*pipTheta+0.07365) )) + (0.003407*pipTheta*pipTheta+-0.5499*pipTheta+68.9) );
    }
    else if(component == 9){   
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (6.325e-04*pipTheta*pipTheta+-0.1008*pipTheta+-2.522) * pow( pipP, (-4.820e-05*pipTheta*pipTheta+0.00698*pipTheta+0.0523) )) + (0.004262*pipTheta*pipTheta+-0.5979*pipTheta+75.75) );
    }
    else if(component == 10){
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (4.371e-04*pipTheta*pipTheta+-0.08361*pipTheta+-2.959) * pow( pipP, (-1.919e-05*pipTheta*pipTheta+0.003797*pipTheta+0.1503) )) + (0.001185*pipTheta*pipTheta+-0.2597*pipTheta+73.94) );
    }
    else if(component == 11){   
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.779e-04*pipTheta*pipTheta+-0.08476*pipTheta+-2.797) * pow( pipP, (-2.150e-05*pipTheta*pipTheta+0.004911*pipTheta+0.106) )) + (7.806e-04*pipTheta*pipTheta+-0.2571*pipTheta+82.2) );
    }
    else if(component == 12){   
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.367e-04*pipTheta*pipTheta+-0.08206*pipTheta+-2.894) * pow( pipP, (-2.620e-05*pipTheta*pipTheta+0.005751*pipTheta+0.08499) )) + (7.258e-04*pipTheta*pipTheta+-0.2592*pipTheta+89.51) );
    }
    else if(component == 13){   
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (5.200e-04*pipTheta*pipTheta+-0.09301*pipTheta+-2.712) * pow( pipP, (-2.873e-05*pipTheta*pipTheta+0.004892*pipTheta+0.1231) )) + (0.001751*pipTheta*pipTheta+-0.2965*pipTheta+97.26) );
    }
    else if(component == 14){   
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.189e-04*pipTheta*pipTheta+-0.07115*pipTheta+-3.461) * pow( pipP, (-6.385e-06*pipTheta*pipTheta+0.001971*pipTheta+0.2394) )) + (6.850e-05*pipTheta*pipTheta+-0.1016*pipTheta+97.85) );
    }
    else if(component == 15){   
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (6.391e-05*pipTheta*pipTheta+-0.05454*pipTheta+-3.61) * pow( pipP, (3.099e-06*pipTheta*pipTheta+0.002311*pipTheta+0.1873) )) + (2.847e-04*pipTheta*pipTheta+-0.214*pipTheta+110.1) );
    }
    else if(component == 16){
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (2.549e-04*pipTheta*pipTheta+-0.06706*pipTheta+-3.343) * pow( pipP, (-4.955e-06*pipTheta*pipTheta+0.002385*pipTheta+0.1911) )) + (6.827e-04*pipTheta*pipTheta+-0.234*pipTheta+118.5) );
    }   
    else if(component == 17){   
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.122e-04*pipTheta*pipTheta+-0.06667*pipTheta+-3.511) * pow( pipP, (-1.559e-06*pipTheta*pipTheta+9.241e-04*pipTheta+0.2608) )) + (7.799e-04*pipTheta*pipTheta+-0.1943*pipTheta+123.7) );
    }
    else if(component == 18){   
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.487e-04*pipTheta*pipTheta+-0.07459*pipTheta+-3.172) * pow( pipP, (-2.213e-05*pipTheta*pipTheta+0.004327*pipTheta+0.133) )) + (0.001043*pipTheta*pipTheta+-0.2602*pipTheta+134.5) );
    }
    else if(component == 19){   
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.784e-04*pipTheta*pipTheta+-0.09061*pipTheta+-2.696) * pow( pipP, (-2.703e-05*pipTheta*pipTheta+0.005993*pipTheta+0.0908) )) + (0.001126*pipTheta*pipTheta+-0.3125*pipTheta+143.1) );
    }
    else if(component == 20){
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (6.098e-04*pipTheta*pipTheta+-0.1127*pipTheta+-2.284) * pow( pipP, (-5.144e-05*pipTheta*pipTheta+0.0086*pipTheta+0.02868) )) + (0.002147*pipTheta*pipTheta+-0.3846*pipTheta+150.2) );
    }
    else if(component == 21){
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (2.482e-04*pipTheta*pipTheta+-0.05894*pipTheta+-3.64) * pow( pipP, (3.361e-06*pipTheta*pipTheta+4.542e-04*pipTheta+0.2431) )) + (0.001738*pipTheta*pipTheta+-0.3876*pipTheta+161.1) );
    }
    else if(component == 22){
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.301e-04*pipTheta*pipTheta+-0.07959*pipTheta+-3.047) * pow( pipP, (-4.136e-06*pipTheta*pipTheta+0.002628*pipTheta+0.1877) )) + (6.880e-04*pipTheta*pipTheta+-0.2405*pipTheta+163.5) );
    }
    else if(component == 23){
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (5.416e-04*pipTheta*pipTheta+-0.08121*pipTheta+-3.357) * pow( pipP, (-2.010e-05*pipTheta*pipTheta+0.001959*pipTheta+0.241) )) + (0.002603*pipTheta*pipTheta+-0.3265*pipTheta+170.9) );
    }
    else if(component == 24){
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (5.171e-04*pipTheta*pipTheta+-0.09415*pipTheta+-2.583) * pow( pipP, (-4.101e-05*pipTheta*pipTheta+0.006931*pipTheta+0.03281) )) + (0.003126*pipTheta*pipTheta+-0.5224*pipTheta+187.9) );
    }
    else if(component == 25){
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (4.246e-04*pipTheta*pipTheta+-0.0711*pipTheta+-3.406) * pow( pipP, (-1.897e-05*pipTheta*pipTheta+0.002847*pipTheta+0.1734) )) + (0.002254*pipTheta*pipTheta+-0.33*pipTheta+188.2) );
    }
    else if(component == 26){
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (2.442e-04*pipTheta*pipTheta+-0.0497*pipTheta+-3.896) * pow( pipP, (-3.200e-06*pipTheta*pipTheta+0.001001*pipTheta+0.214) )) + (1.411e-04*pipTheta*pipTheta+-0.08413*pipTheta+190.3) );
    }
    else if(component == 27){
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.241e-04*pipTheta*pipTheta+-0.06761*pipTheta+-3.338) * pow( pipP, (-1.577e-05*pipTheta*pipTheta+0.003417*pipTheta+0.1435) )) + (0.00114*pipTheta*pipTheta+-0.2615*pipTheta+203.2) );
    }
    else if(component == 28){
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.569e-04*pipTheta*pipTheta+-0.07065*pipTheta+-3.368) * pow( pipP, (-2.469e-05*pipTheta*pipTheta+0.004639*pipTheta+0.1161) )) + (4.478e-04*pipTheta*pipTheta+-0.1482*pipTheta+205.9) );
    }
    else if(component == 29){
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.804e-04*pipTheta*pipTheta+-0.07515*pipTheta+-3.258) * pow( pipP, (-3.277e-05*pipTheta*pipTheta+0.00548*pipTheta+0.1004) )) + (0.001172*pipTheta*pipTheta+-0.2554*pipTheta+216.6) );
    }
    else if(component == 30){
        pipPhis = pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.885e-04*pipTheta*pipTheta+-0.07359*pipTheta+-3.43) * pow( pipP, (-3.157e-06*pipTheta*pipTheta+8.044e-04*pipTheta+0.2732) )) + (0.00103*pipTheta*pipTheta+-0.2171*pipTheta+221.8) );
    }
    else if(component == 31){
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.166e-04*pipTheta*pipTheta+-0.06803*pipTheta+-3.399) * pow( pipP, (-1.460e-05*pipTheta*pipTheta+0.00314*pipTheta+0.1634) )) + (9.521e-04*pipTheta*pipTheta+-0.2423*pipTheta+231.8)));
    }
    else if(component == 32){   
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (4.437e-04*pipTheta*pipTheta+-0.0826*pipTheta+-2.997) * pow( pipP, (-2.490e-05*pipTheta*pipTheta+0.004418*pipTheta+0.1266) )) + (0.002071*pipTheta*pipTheta+-0.373*pipTheta+243.0)));
    }
    else if(component == 33){
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (4.081e-04*pipTheta*pipTheta+-0.08104*pipTheta+-2.983) * pow( pipP, (-1.861e-05*pipTheta*pipTheta+0.003915*pipTheta+0.1358) )) + (0.001467*pipTheta*pipTheta+-0.3164*pipTheta+249.3)));
    }    
    else if(component == 34){    
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (4.715e-04*pipTheta*pipTheta+-0.09383*pipTheta+-2.546) * pow( pipP, (-4.431e-05*pipTheta*pipTheta+0.007695*pipTheta+0.0196) )) + (0.002528*pipTheta*pipTheta+-0.4864*pipTheta+262.9)));
    }
    else if(component == 35){   
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (4.300e-04*pipTheta*pipTheta+-0.08484*pipTheta+-2.948) * pow( pipP, (-3.350e-05*pipTheta*pipTheta+0.00581*pipTheta+0.09291) )) + (0.002028*pipTheta*pipTheta+-0.4035*pipTheta+266.8)));
    }
    else if(component == 36){   
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (4.992e-04*pipTheta*pipTheta+-0.1007*pipTheta+-2.625) * pow( pipP, (-3.374e-05*pipTheta*pipTheta+0.006175*pipTheta+0.09927) )) + (0.002703*pipTheta*pipTheta+-0.4823*pipTheta+273.5)));
    }
    else if(component == 37){    
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (1.146e-04*pipTheta*pipTheta+-0.03712*pipTheta+-4.067) * pow( pipP, (2.812e-06*pipTheta*pipTheta+6.948e-04*pipTheta+0.2039) )) + (-0.001827*pipTheta*pipTheta+0.1166*pipTheta+270.1)));
    }
    else if(component == 38){   
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (8.232e-04*pipTheta*pipTheta+-0.14*pipTheta+-1.462) * pow( pipP, (-5.720e-05*pipTheta*pipTheta+0.009458*pipTheta+0.002741) )) + (0.003422*pipTheta*pipTheta+-0.5596*pipTheta+292.6)));
    }
    else if(component == 39){   
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.125e-04*pipTheta*pipTheta+-0.05256*pipTheta+-3.921) * pow( pipP, (-8.354e-06*pipTheta*pipTheta+9.204e-04*pipTheta+0.2157) )) + (-2.562e-04*pipTheta*pipTheta+-0.01079*pipTheta+286.3)));
    }
    else if(component == 40){
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (4.568e-04*pipTheta*pipTheta+-0.07657*pipTheta+-3.211) * pow( pipP, (-2.475e-05*pipTheta*pipTheta+0.003706*pipTheta+0.1416) )) + (0.001796*pipTheta*pipTheta+-0.2838*pipTheta+300.9)));
    }
    else if(component == 41){   
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (4.247e-04*pipTheta*pipTheta+-0.08127*pipTheta+-2.997) * pow( pipP, (-2.730e-05*pipTheta*pipTheta+0.004874*pipTheta+0.1053) )) + (0.001769*pipTheta*pipTheta+-0.3425*pipTheta+310.5)));
    }
    else if(component == 42){
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (2.548e-04*pipTheta*pipTheta+-0.05479*pipTheta+-3.749) * pow( pipP, (-1.840e-06*pipTheta*pipTheta+0.001161*pipTheta+0.211) )) + (4.092e-04*pipTheta*pipTheta+-0.145*pipTheta+312.6)));
    }
    else if(component == 43){   
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (2.917e-04*pipTheta*pipTheta+-0.06175*pipTheta+-3.568) * pow( pipP, (-1.028e-05*pipTheta*pipTheta+0.002379*pipTheta+0.1845) )) + (4.200e-04*pipTheta*pipTheta+-0.1567*pipTheta+320.0)));
    }
    else if(component == 44){   
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.496e-04*pipTheta*pipTheta+-0.06838*pipTheta+-3.351) * pow( pipP, (-1.405e-05*pipTheta*pipTheta+0.003025*pipTheta+0.1553) )) + (0.001026*pipTheta*pipTheta+-0.2368*pipTheta+330.1)));
    }
    else if(component == 45){   
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (4.105e-04*pipTheta*pipTheta+-0.07559*pipTheta+-3.231) * pow( pipP, (-2.346e-05*pipTheta*pipTheta+0.003959*pipTheta+0.144) )) + (0.001657*pipTheta*pipTheta+-0.3037*pipTheta+338.5)));
    }
    else if(component == 46){   
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.828e-04*pipTheta*pipTheta+-0.07641*pipTheta+-3.17) * pow( pipP, (-2.098e-05*pipTheta*pipTheta+0.004043*pipTheta+0.1399) )) + (0.001148*pipTheta*pipTheta+-0.2726*pipTheta+345.1)));
    }
    else if(component == 47){   
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (3.867e-04*pipTheta*pipTheta+-0.07497*pipTheta+-3.173) * pow( pipP, (-1.908e-05*pipTheta*pipTheta+0.003626*pipTheta+0.1447) )) + (9.592e-04*pipTheta*pipTheta+-0.236*pipTheta+351.7)));
    }
    else if(component == 48){   
        pipPhis = (pipPhiw - ((-1.320e-14*pipTheta*pipTheta+2.058e-12*pipTheta+-3.5e+03) * exp( (4.593e-04*pipTheta*pipTheta+-0.08187*pipTheta+-2.996) * pow( pipP, (-3.379e-05*pipTheta*pipTheta+0.005417*pipTheta+0.09252) )) + (0.002424*pipTheta*pipTheta+-0.4036*pipTheta+363.8)));
    }
    else{
        pipPhis = pipPhiw;
    }
    return pipPhis;
""")

In [5]:
rdf = rdf.Define("WM","""
    //auto elc = ROOT::Math::PxPyPzMVector(ex, ey, ez, 0);
    //auto beam = ROOT::Math::PxPyPzMVector(0, 0, 10.1998, 0);
    //auto targ = ROOT::Math::PxPyPzMVector(0, 0, 0, 0.938);
    auto Output_Vectors = beam + targ - elc;
    return Output_Vectors.M();
""")


In [6]:
e_rdf = rdf.Filter(Calculated_Exclusive_Cuts)
# e_rdf = rdf.Filter("component > 40")

In [7]:
output_file = "/w/hallb-scshelf2102/clas12/ricardos/mcR/phi_shift_analysis/indiv_paddle_shits/skimmed_central_all_phi_shifted.root"
e_rdf.Snapshot("h22", output_file)
del e_rdf

file.Close()